# Initialization & Methods

In [2]:
# Uncomment to INSTALL influxdb the FIRST TIME
#!conda install -c mcrot influxdb

Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - influxdb

Current channels:

  - https://conda.anaconda.org/mcrot/osx-64
  - https://conda.anaconda.org/mcrot/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/free/osx-64
  - https://repo.anaconda.com/pkgs/free/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/pro/osx-64
  - https://repo.anaconda.com/pkgs/pro/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [1]:
# Plot Inline
%matplotlib inline

# Acquisition & Store

### INPUT : REST API

In [2]:
import json
import requests

def getJsonUsingParams(url_template, **kwargs):
    return getJson(url_template.format(kwargs))

def getJson(url):
    # Fetch the URL & get JSON data
    print("Fetching URL : " + url)
    response = requests.get(url)
    jsondata = response.json()
    return jsondata

In [3]:
# TESTING : To run DEMO, look at Notebook => crawling-n-store-toolbox-demo.ipynb
#jsonData = getJson('https://raw.githubusercontent.com/Freeboard/freeboard/master/examples/weather.json')
#jsonData

Fetching URL : https://raw.githubusercontent.com/Freeboard/freeboard/master/examples/weather.json


{'header_image': '',
 'allow_edit': True,
 'panes': [{'title': 'Wind',
   'width': 1,
   'row': {'3': 1},
   'col': {'3': 2},
   'widgets': [{'type': 'pointer',
     'settings': {'direction': 'datasources.Weather.wind_direction',
      'value_text': 'var dir = datasources.Weather.wind_direction;\n\nif(dir <= 22.5)\nreturn "N";\nelse if(dir <= 67.5)\nreturn "NE";\nelse if(dir <= 112.5)\nreturn "E";\nelse if(dir <= 157.5)\nreturn "SE";\nelse if(dir <= 202.5)\nreturn "S";\nelse if(dir <= 247.5)\nreturn "SW";\nelse if(dir <= 292.5)\nreturn "W";\nelse if(dir <= 337.5)\nreturn "NW";\nelse if(dir <= 360)\nreturn "N";'}},
    {'type': 'text_widget',
     'settings': {'size': 'regular',
      'value': 'datasources.Weather.wind_speed',
      'sparkline': True,
      'animate': True,
      'units': 'MPH'}}]},
  {'width': 1,
   'row': {'3': 5},
   'col': {'3': 3},
   'widgets': [{'type': 'text_widget',
     'settings': {'title': 'Sunrise',
      'size': 'regular',
      'value': 'datasources.Weath

### INPUT & OUTPUT : File

In [3]:
import json

def storeData(path, content):
    """Store JSON pretty printed. See http://docs.python.org/library/json.html for more info."""
    with open(path, 'w') as file:
        file.write(json.dumps(content, indent=4))

In [6]:
# TESTING : To run DEMO, look at Notebook => crawling-n-store-toolbox-demo.ipynb
# storeData('serie.json', jsonData)

In [4]:
import json

def readData(path):
    """See http://docs.python.org/library/json.html for more info."""
    with open(path, 'r') as file:
        content = json.load(file)
        return content

In [9]:
# TESTING : To run DEMO, look at Notebook => crawling-n-store-toolbox-demo.ipynb
#jsonData = readData('serie.json')
#jsonData

{'header_image': '',
 'allow_edit': True,
 'panes': [{'title': 'Wind',
   'width': 1,
   'row': {'3': 1},
   'col': {'3': 2},
   'widgets': [{'type': 'pointer',
     'settings': {'direction': 'datasources.Weather.wind_direction',
      'value_text': 'var dir = datasources.Weather.wind_direction;\n\nif(dir <= 22.5)\nreturn "N";\nelse if(dir <= 67.5)\nreturn "NE";\nelse if(dir <= 112.5)\nreturn "E";\nelse if(dir <= 157.5)\nreturn "SE";\nelse if(dir <= 202.5)\nreturn "S";\nelse if(dir <= 247.5)\nreturn "SW";\nelse if(dir <= 292.5)\nreturn "W";\nelse if(dir <= 337.5)\nreturn "NW";\nelse if(dir <= 360)\nreturn "N";'}},
    {'type': 'text_widget',
     'settings': {'size': 'regular',
      'value': 'datasources.Weather.wind_speed',
      'sparkline': True,
      'animate': True,
      'units': 'MPH'}}]},
  {'width': 1,
   'row': {'3': 5},
   'col': {'3': 3},
   'widgets': [{'type': 'text_widget',
     'settings': {'title': 'Sunrise',
      'size': 'regular',
      'value': 'datasources.Weath

### INPUT & OUTPUT : InfluxDB

In [5]:
# http://influxdb-python.readthedocs.io/en/latest/examples.html#tutorials-pandas
from influxdb import DataFrameClient

def storeInflux(db_name, table_name, df, host='localhost', port=8086, **kwargs):
    """Instantiate the connection to the InfluxDB client."""
    user = 'root'
    password = 'root'
    
    # Temporarily avoid line protocol time conversion issues #412, #426, #431.
    protocol = 'json'

    client = DataFrameClient(host, port, user, password, db_name)

    #print("Write DataFrame:{}".format(kwargs['cluster']))
    client.write_points(df, table_name, kwargs, protocol=protocol)

    print("DONE!")

In [ ]:
# TESTING : To run DEMO, look at Notebook => crawling-n-store-toolbox-demo.ipynb
# Also need to create the database 'dbname' using bash and influx
# Use the Docker instance of InfluxDB & Grafana at : https://github.com/frtu/governance-toolbox/tree/master/timeseries
#storeInflux('dbname', dataframe)
#or
#storeInflux('dbname', dataframe, tags=tag_value)

# Processing

In [6]:
from pandas.io.json import json_normalize

def fetchTimeSerie(url_template, json_data_tag='json_tag_containing_data', value_column_name='value', **kwargs):
    url = url_template.format(**kwargs)
    json = getJson(url)

    # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
    df = json_normalize(json, json_value_tag)

    # Move timestamp to index
    # Delete the original timestamp column
    df.index = parseTime(df[0])
    df = df.drop(df.columns[[0]], axis=1)    

    df.columns = [value_column_name]
    return df

def parseTime(timeColumn, timeunit='s', timezone='US/Pacific'):
    """Parse a column from a DataFrame : From epoch time to DatetimeIndex"""
    timeserie = pd.to_datetime(timeColumn.tolist(), unit=timeunit)
    timeserie.tz_localize('UTC').tz_convert(timezone)
    timeserie.name = "Time"
    
    return timeserie

In [7]:
import numpy as np
import pandas as pd

def buildTimeRange(start_date, periods=1, frequency='MS', date_format='%Y/%m/%d-%H:%M:%S.000Z'):
    """frequency : str or DateOffset, default 'D' (calendar daily)
    Frequency strings can have multiples, e.g. '5H'. See
    :ref:`here <timeseries.offset_aliases>` for a list of
    frequency aliases.
    """
    dates = pd.date_range(start_date, periods=(periods + 1), freq=frequency)
    
    # https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
    list_dates = []
    try:
        for date in dates:
            list_dates.append(date.strftime(date_format))
        return list_dates
    except Exception as ex:
        return pd.NaT

In [3]:
# TESTING : To run DEMO, look at Notebook => crawling-n-store-toolbox-demo.ipynb
#buildTimeRange('2018-04-01', 2, 'MS'), buildTimeRange('2018-04-01', 2, 'D'), buildTimeRange('2018-07-01', 30, '12H')

(['2018/04/01-00:00:00.000Z',
  '2018/05/01-00:00:00.000Z',
  '2018/06/01-00:00:00.000Z'],
 ['2018/04/01-00:00:00.000Z',
  '2018/04/02-00:00:00.000Z',
  '2018/04/03-00:00:00.000Z'],
 ['2018/07/01-00:00:00.000Z',
  '2018/07/01-12:00:00.000Z',
  '2018/07/02-00:00:00.000Z',
  '2018/07/02-12:00:00.000Z',
  '2018/07/03-00:00:00.000Z',
  '2018/07/03-12:00:00.000Z',
  '2018/07/04-00:00:00.000Z',
  '2018/07/04-12:00:00.000Z',
  '2018/07/05-00:00:00.000Z',
  '2018/07/05-12:00:00.000Z',
  '2018/07/06-00:00:00.000Z',
  '2018/07/06-12:00:00.000Z',
  '2018/07/07-00:00:00.000Z',
  '2018/07/07-12:00:00.000Z',
  '2018/07/08-00:00:00.000Z',
  '2018/07/08-12:00:00.000Z',
  '2018/07/09-00:00:00.000Z',
  '2018/07/09-12:00:00.000Z',
  '2018/07/10-00:00:00.000Z',
  '2018/07/10-12:00:00.000Z',
  '2018/07/11-00:00:00.000Z',
  '2018/07/11-12:00:00.000Z',
  '2018/07/12-00:00:00.000Z',
  '2018/07/12-12:00:00.000Z',
  '2018/07/13-00:00:00.000Z',
  '2018/07/13-12:00:00.000Z',
  '2018/07/14-00:00:00.000Z',
  '2018/

In [15]:
# TESTING : To run DEMO, look at Notebook => crawling-n-store-toolbox-demo.ipynb
# Create time batches
#time_range = buildTimeRange('2018-07-01', 4, '12H')
#for i in range(len(time_range)):
#    if i != 0:
#        print(time_range[i-1], '=>', time_range[i])

2018/07/01-00:00:00.000Z => 2018/07/01-12:00:00.000Z
2018/07/01-12:00:00.000Z => 2018/07/02-00:00:00.000Z
2018/07/02-00:00:00.000Z => 2018/07/02-12:00:00.000Z
2018/07/02-12:00:00.000Z => 2018/07/03-00:00:00.000Z
